Este notebook apunta a realizar los análisis de datos necesarios para responder a la pregunta siguiente: 

- ¿Cuáles son las caractéristicas sociales de las personas que hablan sobre el cambio climático en la prensa en terminos de "edad", "genero", "profesion" y "popularidad"? 

- ¿Cómo evolucionan estas características en el tiempo? 

- ¿Se puede observar tendencias distintas en los paises siguientes: Costa Rica, España, Chile

ver más detalles en el documento borrador: https://docs.google.com/document/d/1LyIBSMz_Suhy7esHSQt49P56asrhoeCb18qbpOggCY8/edit?usp=sharing

# 1- Construcción del corpus de noticias

In [1]:
import pandas as pd
from pandasql import sqldf

In [5]:
file_corpus_costarica="cambio_climatico_costarica_2018-01-01_2021-12-31.csv"

In [6]:
corpus_costarica=pd.read_csv(file_corpus_costarica)

In [7]:
corpus_costarica

,Unnamed: 0,id_news,country,media_outlet,url,title,text,date,search
0,0,14816148.0,costarica,larepublica,https://www.larepublica.net/noticia/asi-se-abo...,Así se aborda el cambio climático en las escue...,El cambio climático no es visto como una teorí...,2019-05-12,cambio climático
1,1,14760690.0,costarica,larepublica,https://www.larepublica.net/noticia/mas-de-700...,Más de 700 fincas ganaderas se adaptan al camb...,En Costa Rica hay más de 700 fincas ganaderas ...,2019-08-06,cambio climático
2,2,14335379.0,costarica,larepublica,https://www.larepublica.net/noticia/municipali...,Municipalidades se capacitaron en herramientas...,Personal clave de 20 municipalidades se capaci...,2020-12-13,cambio climático
3,3,12610230.0,costarica,larepublica,https://www.larepublica.net/noticia/premio-nac...,Premio Nacional de Ciencia: “Hay efectos irrev...,El fenómeno climático El Niño fue el que condu...,2018-12-12,cambio climático
4,4,14769067.0,costarica,larepublica,https://www.larepublica.net/noticia/fincas-de-...,Fincas de Guanacaste se adaptan al cambio clim...,Una inversión de ¢620 millones permitirá que u...,2019-07-22,cambio climático
...,...,...,...,...,...,...,...,...,...
1681,1685,16386947.0,costarica,elpais,https://www.elpais.cr/2018/06/01/esta-semana-e...,Esta Semana en la Universidad de Costa Rica. D...,INVITACIÓN. ACTIVIDADES DE LA INAUGURACIÓN DE ...,2018-06-01,cambio climático
1682,1686,13472869.0,costarica,nacion,https://www.nacion.com/el-pais/servicios/a-50-...,A 50 años de la tragedia del volcán Arenal: el...,“Yo vivo tranquila. El volcán no me asusta. Pa...,2018-07-22,cambio climático
1683,1687,13365281.0,costarica,nacion,https://www.nacion.com/el-pais/carlos-alvarado...,¿Cuáles son los primeros beneficios que los co...,Los rostros de los huelguistas que lo increpar...,2018-10-05,cambio climático
1684,1688,13564302.0,costarica,nacion,https://www.nacion.com/el-pais/politica/en-viv...,Carlos Alvarado: 'Juro ser la mejor versión de...,Carlos Alvarado Quesada se convirtió esta maña...,2018-05-08,cambio climático


- ¿Cuáles son los medios y su número de noticias?

In [8]:
query="""SELECT media_outlet,count(*) as noticias, min(date),max(date)
FROM corpus_costarica GROUP BY media_outlet ORDER BY noticias DESC;"""
result=sqldf(query)
result

,media_outlet,noticias,min(date),max(date)
0,elpais,742,2018-01-08,2021-04-24
1,nacion,490,2018-01-09,2021-04-25
2,larepublica,355,2018-06-11,2021-04-26
3,lateja,99,2018-01-11,2021-03-31


- ¿Cómo se distribuyen las noticias en el tiempo?

In [10]:
#Contar el número de documentos por año

query="""SELECT strftime('%Y', date) as year,count(*) as documents
FROM corpus_costarica GROUP BY year;"""
result=sqldf(query)
result

,year,documents
0,2018,349
1,2019,655
2,2020,496
3,2021,186


# 2. Detección de las entidades de tipo Personas

In [11]:
from collections import Counter
import spacy
nlp = spacy.load("es_core_news_lg") #cambiar por "es_core_news_lg" para el análisis final

In [14]:
df_persons = pd.DataFrame(columns=["date", "name"])

for index, row in corpus_costarica.iterrows():
    print(index)
    text = str(row["text"])
    date = row["date"]
    person_in_text = []
    
    if(len(text)>0):
        doc = nlp(text)
        
        for ent in doc.ents:
            if (ent.label_ == "PER"):
                person = ent.text
                if (' ' in person) == True:
                    if person not in person_in_text:
                        df_persons = df_persons.append({"date": date,"name":  person}, ignore_index=True)
                        person_in_text.append(person)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
df_persons

,date,name
0,2019-05-12,Alexandra Ulate
1,2019-05-12,Desarrollo Curricular
2,2019-08-06,Ana Cristina Quirós
3,2020-12-13,Andrea Meza
4,2018-12-12,Hugo Hidalgo
...,...,...
7183,2020-12-03,Y Conape
7184,2020-12-03,Víctor Morales Mora
7185,2020-12-03,Carlos Ricardo Benavides
7186,2020-12-03,Sala IV


In [16]:
query="""SELECT strftime('%Y', date) as year,name, count(*) as freq
FROM df_persons GROUP BY year, name ORDER BY count(*) DESC;"""
df_persons_by_year=sqldf(query)
df_persons_by_year

,year,name,freq
0,2019,Carlos Alvarado,84
1,2018,Donald Trump,68
2,2018,Carlos Alvarado,66
3,2020,Donald Trump,66
4,2019,Carlos Manuel Rodríguez,62
...,...,...,...
4440,2021,«Kim Stanley Robinson,1
4441,2021,Ángela Merkel,1
4442,2021,Óscar Avalle,1
4443,2021,Óscar Castillo,1


# 3. Extracción de metadatos sobre cada persona (género, popularidad, edad, profesión)

- GENERO

In [17]:
import unidecode

DICT_FIRSTNAMES="names_gender.csv"

gen_nam=pd.read_csv(DICT_FIRSTNAMES)

In [18]:
def get_gender(named_entity, language):
    #We consider the first token  of the named entity as the firstname
    firstname=named_entity.split(' ', maxsplit=1)[0] 
    firstname_upper= firstname.upper()
    firstname_unidecode_upper=unidecode.unidecode(firstname).upper()

    gender_df = gen_nam[(gen_nam.firstname==firstname) | (gen_nam.firstname==firstname_upper) | (gen_nam.firstname==firstname_unidecode_upper)]
    male_df = gender_df[gender_df.gender=='M']
    female_df = gender_df[gender_df.gender=='F']
    if(len(gender_df)!=0):
        if(len(female_df)/len(gender_df) >= 0.6  ):
            return female_df.values[0][2]
        if(len(male_df)/len(gender_df) >= 0.6 ):
            return male_df.values[0][2]
        return 'N'    
    return None

In [19]:
get_gender("Jamal Khashoggi","Spanish")

'M'

In [20]:
df_persons_by_year["gender"] = ""


In [21]:
df_persons_by_year

,year,name,freq,gender
0,2019,Carlos Alvarado,84,
1,2018,Donald Trump,68,
2,2018,Carlos Alvarado,66,
3,2020,Donald Trump,66,
4,2019,Carlos Manuel Rodríguez,62,
...,...,...,...,...
4440,2021,«Kim Stanley Robinson,1,
4441,2021,Ángela Merkel,1,
4442,2021,Óscar Avalle,1,
4443,2021,Óscar Castillo,1,


In [22]:
for index, row in df_persons_by_year.iterrows():
    print(index)
    gender=get_gender(row["name"],"Spanish")
    df_persons_by_year.loc[index,['gender']] = gender

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703


In [23]:
df_persons_by_year

,year,name,freq,gender
0,2019,Carlos Alvarado,84,M
1,2018,Donald Trump,68,M
2,2018,Carlos Alvarado,66,M
3,2020,Donald Trump,66,M
4,2019,Carlos Manuel Rodríguez,62,M
...,...,...,...,...
4440,2021,«Kim Stanley Robinson,1,None
4441,2021,Ángela Merkel,1,F
4442,2021,Óscar Avalle,1,M
4443,2021,Óscar Castillo,1,M


In [24]:
query="""SELECT year, gender, sum(freq), count(*) as nb_persons
FROM df_persons_by_year GROUP BY year, gender ORDER BY year, gender;"""
result=sqldf(query)
result

,year,gender,sum(freq),nb_persons
0,2018,None,224,181
1,2018,F,339,244
2,2018,M,1080,619
3,2018,N,17,8
4,2019,None,489,324
5,2019,F,643,387
6,2019,M,1584,934
7,2019,N,7,4
8,2020,None,310,236
9,2020,F,481,306


- POPULARIDAD

In [8]:
import statistics
import pageviewapi

In [9]:
def get_popularity(named_entity, date):
    popularity_en=calculate_popularity(named_entity, date,"en")
    popularity_es=calculate_popularity(named_entity, date,"es")
    popularity_fr=calculate_popularity(named_entity, date,"fr")
    popularity_it=calculate_popularity(named_entity, date,"it")
    popularity_pt=calculate_popularity(named_entity, date,"pt")
    return {"popularity_en":popularity_en,"popularity_es":popularity_es,"popularity_fr":popularity_fr,"popularity_it":popularity_it,"popularity_pt":popularity_pt}

In [10]:
import unidecode
from datetime import timedelta, datetime

def calculate_popularity(named_entity, date, language):
    start_date = (date - timedelta(days=365)).strftime('%Y%m%d')
    end_date = (date).strftime('%Y%m%d')
    try: 
        result=pageviewapi.per_article(f'{language}.wikipedia', named_entity, start_date , end_date,
                        access='all-access', agent='all-agents', granularity='daily')    
    except Exception as error:
        return 0
    views = []
    for item in result.items():
        for article in item[1]:
            views.append(article['views'])
    return int(statistics.mean(views))

In [28]:
get_popularity("Gavin Newsom",datetime.now())

{'popularity_en': 8696,
 'popularity_es': 107,
 'popularity_fr': 91,
 'popularity_it': 51,
 'popularity_pt': 45}

In [29]:
df_persons_by_year["popularity_en"] = ""
df_persons_by_year["popularity_es"] = ""
df_persons_by_year["popularity_fr"] = ""
df_persons_by_year["popularity_it"] = ""
df_persons_by_year["popularity_pt"] = ""

In [30]:
 df_persons_by_year

,year,name,freq,gender,popularity_en,popularity_es,popularity_fr,popularity_it,popularity_pt
0,2019,Carlos Alvarado,84,M,,,,,
1,2018,Donald Trump,68,M,,,,,
2,2018,Carlos Alvarado,66,M,,,,,
3,2020,Donald Trump,66,M,,,,,
4,2019,Carlos Manuel Rodríguez,62,M,,,,,
...,...,...,...,...,...,...,...,...,...
4440,2021,«Kim Stanley Robinson,1,None,,,,,
4441,2021,Ángela Merkel,1,F,,,,,
4442,2021,Óscar Avalle,1,M,,,,,
4443,2021,Óscar Castillo,1,M,,,,,


- Obtener textos desde Wikipedia, Edad & Profesión

In [3]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import pipeline

In [4]:
ES_MODEL_LANGUAGE="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
EN_MODEL_LANGUAGE="distilbert-base-cased-distilled-squad"

In [5]:
tokenizer_es_language = AutoTokenizer.from_pretrained(ES_MODEL_LANGUAGE)
model_es_language = AutoModelForQuestionAnswering.from_pretrained(ES_MODEL_LANGUAGE)

tokenizer_en_language = AutoTokenizer.from_pretrained(EN_MODEL_LANGUAGE)
model_en_language = AutoModelForQuestionAnswering.from_pretrained(EN_MODEL_LANGUAGE)

In [6]:
q_a_es = pipeline("question-answering", model=model_es_language, tokenizer=tokenizer_es_language)
q_a_en = pipeline("question-answering", model=model_en_language, tokenizer=tokenizer_en_language)

In [36]:
df_persons_by_year["wikipedia"] = ""
df_persons_by_year["birth"] = ""
df_persons_by_year["profession"] = ""

In [7]:
import wikipedia

In [38]:
df_persons_by_year.to_csv("costarica_persons_cambioclimatico.csv")

In [12]:
df_persons_by_year = pd.read_csv("costarica_persons_cambioclimatico.csv")

In [ ]:
for index, row in df_persons_by_year.iterrows():
        if (index <= 2534):
            continue
        print(index)
        name=row["name"]
        print(name)
        year=row["year"]
        
        ##Popularity
        date = datetime(int(year), 12, 31, 0, 0, 0, 0)
        popularities=get_popularity(name,date)
        
        df_persons_by_year.loc[index,['popularity_en']] = popularities["popularity_en"]
        df_persons_by_year.loc[index,['popularity_es']] = popularities["popularity_es"]
        df_persons_by_year.loc[index,['popularity_fr']] = popularities["popularity_fr"]
        df_persons_by_year.loc[index,['popularity_it']] = popularities["popularity_it"]
        df_persons_by_year.loc[index,['popularity_pt']] = popularities["popularity_pt"]
        
        ##Wikipedia
        lang="en"
        wikipedia.set_lang("en")
        text=""
        try:
            text = wikipedia.summary(name, sentences=3)
        except:
            wikipedia.set_lang("es")
            lang="es"
            try:
                text = wikipedia.summary(name, sentences=3)
            except:
                print(name+" no está en Wikipedia.")
        
        df_persons_by_year.loc[index,['wikipedia']] = text
        
        if text == "":
            df_persons_by_year.loc[index,['birth']] = ""
            df_persons_by_year.loc[index,['profession']] = ""
            continue
        
        ##Edad & Profesión
        if lang == "en":
            result=q_a_en(question="What year was she or he born?", context=text)
            df_persons_by_year.loc[index,['birth']] = result["answer"]
            print(result["answer"])
            result=q_a_en(question="What is her or his profession?", context=text)
            df_persons_by_year.loc[index,['profession']] = result["answer"]
            print(result["answer"])
        else:
            result = q_a_es(question="¿En qué año nació el o ella?", context=text)
            df_persons_by_year.loc[index,['birth']] = result["answer"]
            print(result["answer"])
            result = q_a_es(question="¿Cuál es su profesión?", context=text)
            df_persons_by_year.loc[index,['profession']] = result["answer"]
            print(result["answer"])

2535
María Julia Arana
María Julia Arana no está en Wikipedia.
2536
María Julia Muñoz
1950
doctor in medicine
2537
María Luisa
1751
aristocrats
2538
María Luisa Ramos Urzagaste
2016
Ambassador
2539
María Luisa Ávila ExministraSalud
2019
political leaders
2540
María Maglianesi
900-2000m
breeding
2541
María Noel Vaeza
María Noel Vaeza no está en Wikipedia.
2542
María Vita Monge
1972
abogada
2543
María da Paz
1791
militar
2544
Maríano Figueres
Maríano Figueres no está en Wikipedia.
2545
Mateusz Morawiecki
1968
economist, historian
2546
Matias Spektor
Born in Rosario, Argentina
Associate Professor of International Relations
2547
Mato Grosso
Mato Grosso
Mato Grosso
2548
Matriz Fiscal Verde
1956
economista
2549
Matthias Kleiner
2020
non-university research institutes
2550
Mattias Säfvenberg


/home/mvernier/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


2017
Alpine skiing
2551
Matías Miranda
2000
footballer
2552
Maurice Piat
1941
Mauritian Roman Catholic prelate
2553
Mauricio Chacón
1969
Professional
2554
Mauricio Espinoza Bolaños
Mauricio Espinoza Bolaños no está en Wikipedia.
2555
Maurizio Musmanni PresidenteCámara
Maurizio Musmanni PresidenteCámara no está en Wikipedia.
2556
Maurizio MusmanniPresidenteCámara de la Industria AlimentariaNos
Maurizio MusmanniPresidenteCámara de la Industria AlimentariaNos no está en Wikipedia.
2557
Max Borge Director
Max Borge Director no está en Wikipedia.
2558
Max Peralta
1894
Dr. Maximiliano Peralta Jimenez
2559
Max Peralta de Cartago
Dios
médico
2560
Maxime Bernier
1963
law, finance and banking
2561
Maximiliano Reyes
1960
actor
2562
Melania Guerra
2004
pilot
2563
Melania Guerra Carrillo
Melania Guerra Carrillo no está en Wikipedia.
2564
Meles Zenawi
1955
soldier
2565
Melinda Gates
1964
empresaria
2566
Melvin Monge Director
Melvin Monge Director no está en Wikipedia.
2567
Melvin MongeDirector Gener

/home/mvernier/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/mvernier/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


1971
escritor
2664
Pablo Pascale
Pablo Pascale no está en Wikipedia.
2665
Pablo Vargas CEOGrupo Britt
Pablo Vargas CEOGrupo Britt no está en Wikipedia.
2666
Pablo VargasCEOGrupo BrittEn
Pablo VargasCEOGrupo BrittEn no está en Wikipedia.
2667
Pacas Pai
Pacas Pai no está en Wikipedia.
2668
Pamela Castillo ViceministraAmbiente
Pamela Castillo ViceministraAmbiente no está en Wikipedia.
2669
Paola Vega DiputadaPAC
Costa Rica
Members of the Legislative Assembly of Costa Rica
2670
Paola Vega DiputadaPAC Creo
Paola Vega DiputadaPAC Creo no está en Wikipedia.
2671
Paola Vega DiputadaPAC Lo
Paola Vega DiputadaPAC Lo no está en Wikipedia.
2672
Paola VegaDiputadaPACCreo
Paola VegaDiputadaPACCreo no está en Wikipedia.
2673
Paola VegaDiputadaPACLo
Paola VegaDiputadaPACLo no está en Wikipedia.
2674
Paola VegaDiputadaPACNuestro
Paola VegaDiputadaPACNuestro no está en Wikipedia.
2675
Paola Yáñez
1943
politician
2676
Papagayo Lea
2003
actor, director y guionista
2677
Para Dapena
Para Dapena no está en W

1949
businessman and politician
2826
Secretario General Adjunto de las Naciones Unidas para los Derechos Humanos
1993
responsable de los derechos humanos
2827
Segundo Vicepresidente de la República
República
vicepresidenta constitucional de la República del Perú
2828
Según Bachelet
1951
médica
2829
Según Frei
1942
ingeniero
2830
Según Haydeé Rodríguez
1923
guerrillera
2831
Según Kellner
Según Kellner no está en Wikipedia.
2832
Según Nyusi
Según Nyusi no está en Wikipedia.
2833
Según Scheer
de
de
2834
Según Solís
1954
político y economista
2835
Según Trump
1946
empresario
2836
Senadora Harris
Senadora Harris no está en Wikipedia.
2837
Sergio Pascual
1960
a performer of merengue and bolero
2838
Serguéi Lavrov
1950
Ministro de Asuntos Exteriores de Rusia
2839
Serona Methorst
Serona Methorst no está en Wikipedia.
2840
Señor Padre
eleventh century
El Cantar de mio Cid
2841
Sharada Saraf
Sharada Saraf no está en Wikipedia.
2842
Sharice Davids
1980
attorney
2843
Sheikh Hasina
1947
politician


8 October 1974
economist
2982
Zuzana Èaputová
Zuzana Èaputová no está en Wikipedia.
2983
don Carlos
Giuseppe Verdi
Don Carlo
2984
don Omar
1978
singer, songwriter, rapper, record producer and actor
2985
ministro Alvarado
1980
político, escritor, periodista
2986
pic.twitter.com/MZMoXg6G14 Thunberg
pic.twitter.com/MZMoXg6G14 Thunberg no está en Wikipedia.
2987
von der Leyen
1958
physician
2988
«Costa Rica
Costa Rica
Costa Rica
2989
«El SINAMECC
«El SINAMECC no está en Wikipedia.
2990
«Victoria histórica» Maduro
«Victoria histórica» Maduro no está en Wikipedia.
2991
Álvaro García Linera
1962
sociologist and political theorist
2992
Álvaro Sagot Abogado
Álvaro Sagot Abogado no está en Wikipedia.
2993
Álvaro Sagot Rodríguez
1926
Cartago
2994
Álvaro SagotAbogado
Álvaro SagotAbogado no está en Wikipedia.
2995
Álvaro Sáenz
1957
film and television actor and former model
2996
Ángel Gurría
1950
economist and diplomat
2997
Ángel Víctor Torres
1966
politician
2998
Édgar Mora
2021
History
2999
Érick

1961
periodista
3156
Carlos V de España
1938
Juan Carlos I
3157
Carlota García Encina
1896
maestro, abogado
3158
Carmen Calvo
1957
author
3159
Carolina Ramírez Azofeifa
President of the Legislative Assembly of Costa Rica
President of the Legislative Assembly of Costa Rica is the presiding officer
3160
Carolina Recinos
Carolina Recinos no está en Wikipedia.
3161
Cary Funk
Cary Funk no está en Wikipedia.
3162
Cate Blancjett
Cate Blancjett no está en Wikipedia.
3163
Cayetano Heredia
1797
physician
3164
Cecilia Medina
1935
jurist
3165
Celeste López
1985
model
3166
Celeste López ViceministraGestión Ambiental Quiero
Celeste López ViceministraGestión Ambiental Quiero no está en Wikipedia.
3167
Celeste LópezViceministraGestión AmbientalQuiero
Celeste LópezViceministraGestión AmbientalQuiero no está en Wikipedia.
3168
Chet France
Chet France no está en Wikipedia.
3169
Chimamanda Ngozi Adichie
1977
feminist
3170
Chris Dickman
Chris Dickman no está en Wikipedia.
3171
Chris Turney
Christian S. M. 

In [40]:
df_persons_by_year

,Unnamed: 0,year,name,freq,gender,popularity_en,popularity_es,popularity_fr,popularity_it,popularity_pt,wikipedia,birth,profession
0,0,2019,Carlos Alvarado,84,M,4.0,7.0,29.0,3.0,1.0,Carlos Andrés Alvarado Quesada (Spanish pronun...,14 January 1980,"writer, journalist and political scientist"
1,1,2018,Donald Trump,68,M,51092.0,4651.0,3796.0,1922.0,1871.0,"Donald John Trump (born June 14, 1946) is an A...",1946,media personality
2,2,2018,Carlos Alvarado,66,M,5.0,5.0,42.0,4.0,1.0,Carlos Andrés Alvarado Quesada (Spanish pronun...,14 January 1980,"writer, journalist and political scientist"
3,3,2020,Donald Trump,66,M,617453.0,9681.0,7807.0,4653.0,2666.0,"Donald John Trump (born June 14, 1946) is an A...",1946,media personality
4,4,2019,Carlos Manuel Rodríguez,62,M,1.0,1.0,0.0,1.0,0.0,Carlos Manuel Cecilio Rodríguez Santiago (Nove...,1918,layperson
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4440,4440,2021,«Kim Stanley Robinson,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4441,4441,2021,Ángela Merkel,1,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4442,4442,2021,Óscar Avalle,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4443,4443,2021,Óscar Castillo,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ERROR! Session/line number was not unique in database. History logging moved to new session 218


In [41]:
df_persons_by_year.to_csv("costarica_persons_cambioclimatico_final.csv")

In [2]:
df_persons_by_year = pd.read_csv("costarica_persons_cambioclimatico_final.csv")
df_persons_by_year

,Unnamed: 0,Unnamed: 0.1,year,name,freq,gender,popularity_en,popularity_es,popularity_fr,popularity_it,popularity_pt,wikipedia,birth,profession
0,0,0,2019,Carlos Alvarado,84,M,4.0,7.0,29.0,3.0,1.0,Carlos Andrés Alvarado Quesada (Spanish pronun...,14 January 1980,"writer, journalist and political scientist"
1,1,1,2018,Donald Trump,68,M,51092.0,4651.0,3796.0,1922.0,1871.0,"Donald John Trump (born June 14, 1946) is an A...",1946,media personality
2,2,2,2018,Carlos Alvarado,66,M,5.0,5.0,42.0,4.0,1.0,Carlos Andrés Alvarado Quesada (Spanish pronun...,14 January 1980,"writer, journalist and political scientist"
3,3,3,2020,Donald Trump,66,M,617453.0,9681.0,7807.0,4653.0,2666.0,"Donald John Trump (born June 14, 1946) is an A...",1946,media personality
4,4,4,2019,Carlos Manuel Rodríguez,62,M,1.0,1.0,0.0,1.0,0.0,Carlos Manuel Cecilio Rodríguez Santiago (Nove...,1918,layperson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4440,4440,4440,2021,«Kim Stanley Robinson,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4441,4441,4441,2021,Ángela Merkel,1,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4442,4442,4442,2021,Óscar Avalle,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4443,4443,4443,2021,Óscar Castillo,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
